<a href="https://colab.research.google.com/github/Shahorukh/Student-Grading-and-Performance-Analysis-System/blob/main/BPP_3_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import csv

# Define the initial student data
initial_data = [
    {"Student ID": 1, "Name": "Sharukh", "Subjects": {"Math": 95, "English": 88, "Science": 92}, "Grade": None},
    {"Student ID": 2, "Name": "Khan", "Subjects": {"Math": 78, "English": 85, "Science": 80}, "Grade": None},
    {"Student ID": 3, "Name": "Safa", "Subjects": {"Math": 65, "English": 70, "Science": 72}, "Grade": None},
]

In [30]:
# Create a CSV file with initial data
def initialize_csv_file(filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(["Student ID", "Name", "Subjects", "Marks", "Grade"])
        # Write the data
        for student in initial_data:
            subjects = student["Subjects"]
            marks = ", ".join([f"{subj}: {marks}" for subj, marks in subjects.items()])
            writer.writerow([student["Student ID"], student["Name"], ", ".join(subjects.keys()), marks, student["Grade"]])

# Load data from CSV into Python data structures
def load_data_from_csv(filename):
    data = []
    with open(filename, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            subjects = dict(
                [tuple(subject.split(": ")) for subject in row["Marks"].split(", ")]
            )
            data.append({
                "Student ID": int(row["Student ID"]),
                "Name": row["Name"],
                "Subjects": {k: int(v) for k, v in subjects.items()},
                "Grade": row["Grade"],
            })
    return data

In [31]:
# Add a new student
def add_student(data, student_id, name, subjects):
    data.append({"Student ID": student_id, "Name": name, "Subjects": subjects, "Grade": None})

In [32]:
# Update an existing student's details
def update_student(data, student_id, updated_name=None, updated_subjects=None):
    for student in data:
        if student["Student ID"] == student_id:
            if updated_name:
                student["Name"] = updated_name
            if updated_subjects:
                student["Subjects"].update(updated_subjects)
            return True
    return False

In [33]:
# Remove a student record by ID
def remove_student(data, student_id):
    for i, student in enumerate(data):
        if student["Student ID"] == student_id:
            del data[i]
            return True
    return False

In [34]:
# Calculate grade based on marks
def calculate_grade(average_marks):
    if 90 <= average_marks <= 100:
        return "A"
    elif 80 <= average_marks < 90:
        return "B"
    elif 70 <= average_marks < 80:
        return "C"
    else:
        return "F"


In [35]:
# Assign grades to all students
def assign_grades(data):
    for student in data:
        subjects = student["Subjects"]
        average_marks = sum(subjects.values()) / len(subjects)
        student["Grade"] = calculate_grade(average_marks)

In [36]:
# Search for a student by ID or name
def search_student(data, student_id=None, name=None):
    results = []
    for student in data:
        if (student_id is not None and student["Student ID"] == student_id) or \
           (name is not None and student["Name"].lower() == name.lower()):
            results.append(student)
    return results

In [37]:
# Filter students by grade
def filter_students_by_grade(data, grade):
    return [student for student in data if student["Grade"] == grade]


In [38]:
# Show students who failed in any subject
def students_failed_any_subject(data):
    failed_students = []
    for student in data:
        if any(marks < 70 for marks in student["Subjects"].values()):
            failed_students.append(student)
    return failed_students


In [39]:
# Class performance summary
def class_performance_summary(data):
    total_students = len(data)
    total_marks = sum(sum(student["Subjects"].values()) for student in data)
    total_subjects = sum(len(student["Subjects"]) for student in data)
    average_marks = total_marks / total_subjects

    top_scorer = max(data, key=lambda x: sum(x["Subjects"].values()))
    low_scorer = min(data, key=lambda x: sum(x["Subjects"].values()))

    pass_count = sum(1 for student in data if all(marks >= 70 for marks in student["Subjects"].values()))
    pass_percentage = (pass_count / total_students) * 100

    return {
        "Average Marks": average_marks,
        "Top Scorer": top_scorer,
        "Low Scorer": low_scorer,
        "Pass Percentage": pass_percentage
    }

In [40]:
# Initialize and load the data
csv_filename = "student_data.csv"
initialize_csv_file(csv_filename)
students = load_data_from_csv(csv_filename)

# Assign grades to students
assign_grades(students)

In [41]:
# Example menu-driven interaction
def menu():
    while True:
        print("\nMenu:")
        print("1. Add Student")
        print("2. Update Student")
        print("3. Remove Student")
        print("4. View Class Performance Summary")
        print("5. Search Student")
        print("6. Filter Students by Grade")
        print("7. Show Students Who Failed Any Subject")
        print("8. Exit")

        choice = input("Enter your choice: ")

        if choice == "1":
            student_id = int(input("Enter Student ID: "))
            name = input("Enter Name: ")
            subjects = {}
            for subject in ["Math", "English", "Science"]:
                marks = int(input(f"Enter marks for {subject}: "))
                subjects[subject] = marks
            add_student(students, student_id, name, subjects)
            assign_grades(students)
        elif choice == "2":
            student_id = int(input("Enter Student ID to update: "))
            updated_name = input("Enter new name (leave blank to skip): ")
            updated_subjects = {}
            for subject in ["Math", "English", "Science"]:
                marks = input(f"Enter new marks for {subject} (leave blank to skip): ")
                if marks:
                    updated_subjects[subject] = int(marks)
            update_student(students, student_id, updated_name or None, updated_subjects or None)
            assign_grades(students)
        elif choice == "3":
            student_id = int(input("Enter Student ID to remove: "))
            remove_student(students, student_id)
        elif choice == "4":
            summary = class_performance_summary(students)
            print("Class Performance Summary:")
            print(summary)
        elif choice == "5":
            search_option = input("Search by (1) ID or (2) Name? ")
            if search_option == "1":
                student_id = int(input("Enter Student ID: "))
                print(search_student(students, student_id=student_id))
            elif search_option == "2":
                name = input("Enter Student Name: ")
                print(search_student(students, name=name))
        elif choice == "6":
            grade = input("Enter grade to filter (A/B/C/F): ")
            print(filter_students_by_grade(students, grade))
        elif choice == "7":
            print("Students who failed in any subject:")
            print(students_failed_any_subject(students))
        elif choice == "8":
            print("Exiting program.")
            break
        else:
            print("Invalid choice. Please try again.")

In [43]:
check_project=menu()


Menu:
1. Add Student
2. Update Student
3. Remove Student
4. View Class Performance Summary
5. Search Student
6. Filter Students by Grade
7. Show Students Who Failed Any Subject
8. Exit
Enter your choice: 1
Enter Student ID: 4
Enter Name: Shakil
Enter marks for Math: 60
Enter marks for English: 70
Enter marks for Science: 75

Menu:
1. Add Student
2. Update Student
3. Remove Student
4. View Class Performance Summary
5. Search Student
6. Filter Students by Grade
7. Show Students Who Failed Any Subject
8. Exit
Enter your choice: 1
Enter Student ID: 5
Enter Name: Safayet
Enter marks for Math: 95
Enter marks for English: 98
Enter marks for Science: 97

Menu:
1. Add Student
2. Update Student
3. Remove Student
4. View Class Performance Summary
5. Search Student
6. Filter Students by Grade
7. Show Students Who Failed Any Subject
8. Exit
Enter your choice: 2
Enter Student ID to update: 1
Enter new name (leave blank to skip): Shahorukh
Enter new marks for Math (leave blank to skip): 100
Enter ne